In [1]:
import random
import time
import datetime
import numpy as np
from numpy import array
import pandas as pd
from pydub import AudioSegment
import os, shutil, glob
import gc
import sys
import h5py
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

#imported for testing
import wave
import contextlib

# for outputing file
import scipy.stats.stats as st

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from scipy.cluster.vq import vq, kmeans, whiten
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.impute import SimpleImputer
import pickle
import librosa
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from pandas.plotting import parallel_coordinates
import pickle

from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten
from keras.constraints import maxnorm
from keras.layers import Add
from keras.models import Sequential
from keras.layers import Dense, Add
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout, Input
from keras.callbacks import EarlyStopping
from keras.layers import Activation
from keras.utils import np_utils
from keras.utils import to_categorical

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

sample_rate=44100
hop_length = 441  # frame size= 2*hop
segment_length=int(sample_rate*0.2)  #0.2
segment_pad=int(sample_rate*0.02)     #0.02
overlapping=int(sample_rate*0.1)   #0.1

NumofFeaturetoUse = 272 # this will re-assigned for different classifiers
frame_number = 48

try:
    NumofFeaturetoUse = int(sys.argv[1])
    print('Number of features to use is set to ' + str(sys.argv[1]) )
except:
    print('Number of features are unspecified. Defaut is set to = 272.')

Using TensorFlow backend.


Number of features are unspecified. Defaut is set to = 272.


In [2]:
h_directory = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Happy//'
h_feature_vector = 'Features_48//h_feature_vector_48.npy' 
h_label_vector = 'Features_48//h_label_vector_48.npy'
h_label = 'H'

a_directory = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Angry//'
a_feature_vector = 'Features_48//a_feature_vector_48.npy' 
a_label_vector = 'Features_48//a_label_vector_48.npy'
a_label = 'A'

n_directory = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Neutral//'
n_feature_vector = 'Features_48//n_feature_vector_48.npy' 
n_label_vector = 'Features_48//n_label_vector_48.npy'
n_label = 'N'

s_directory = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Sad//'
s_feature_vector = 'Features_48//s_feature_vector_48.npy' 
s_label_vector = 'Features_48//s_label_vector_48.npy'
s_label = 'S'

h_test = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Happy_test//'
h_feature_vector_test = 'Features_48//h_feature_vector_test_48.npy' 
h_label_vector_test = 'Features_48//h_label_vector_test_48.npy'
h_label_test = 'H'

a_test = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Angry_test//'
a_feature_vector_test = 'Features_48//a_feature_vector_test_48.npy' 
a_label_vector_test = 'Features_48//a_label_vector_test_48.npy'
a_label_test = 'A'

n_test = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Neutral_test//'
n_feature_vector_test = 'Features_48//n_feature_vector_test_48.npy' 
n_label_vector_test = 'Features_48//n_label_vector_test_48.npy'
n_label_test = 'N'

s_test = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Sad_test//'
s_feature_vector_test = 'Features_48//s_feature_vector_test_48.npy' 
s_label_vector_test = 'Features_48//s_label_vector_test_48.npy'
s_label_test = 'S'

In [3]:
def update_progress(progress):
    bar_length = 100
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [4]:
def function_FeatureExtractfromSinglewindow(y,hop_length,sr):

    genFeatures=np.array([])
    try:
        mfcc0 = librosa.feature.mfcc(y=y, sr=sr, n_fft=hop_length*2, hop_length=hop_length, n_mfcc=13)
        mfcc=np.transpose(mfcc0)

        genFeatures = np.hstack((genFeatures, np.amin(mfcc, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(mfcc, 0)))
        genFeatures = np.hstack((genFeatures, np.median(mfcc, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(mfcc, 0)))
        genFeatures = np.hstack((genFeatures, np.std(mfcc, 0)))
        genFeatures = np.hstack((genFeatures, np.var(mfcc, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(mfcc, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(mfcc, 0)))
        #print(genFeatures.shape)

        mfcc_delta=librosa.feature.delta(mfcc0)
        mfcc_delta=np.transpose(mfcc_delta)
        genFeatures = np.hstack((genFeatures, np.amin(mfcc_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(mfcc_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.median(mfcc_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(mfcc_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.std(mfcc_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.var(mfcc_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(mfcc_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(mfcc_delta, 0)))
        #print(genFeatures.shape)

        zcr0=librosa.feature.zero_crossing_rate(y=y, frame_length=hop_length*2, hop_length=hop_length)
        zcr=np.transpose(zcr0)
        genFeatures = np.hstack((genFeatures, np.amin(zcr, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(zcr, 0)))
        genFeatures = np.hstack((genFeatures, np.median(zcr, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(zcr, 0)))
        genFeatures = np.hstack((genFeatures, np.std(zcr, 0)))
        genFeatures = np.hstack((genFeatures, np.var(zcr, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(zcr, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(zcr, 0)))
        #print(genFeatures.shape)

        zcr_delta=librosa.feature.delta(zcr0)
        zcr_delta=np.transpose(zcr_delta)
        genFeatures = np.hstack((genFeatures, np.amin(zcr_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(zcr_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.median(zcr_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(zcr_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.std(zcr_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.var(zcr_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(zcr_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(zcr_delta, 0)))
        #print(genFeatures.shape)

        Erms0=librosa.feature.rms(y=y, frame_length=hop_length*2, hop_length=hop_length)
        Erms=np.transpose(Erms0)
        genFeatures = np.hstack((genFeatures, np.amin(Erms, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(Erms, 0)))
        genFeatures = np.hstack((genFeatures, np.median(Erms, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(Erms, 0)))
        genFeatures = np.hstack((genFeatures, np.std(Erms, 0)))
        genFeatures = np.hstack((genFeatures, np.var(Erms, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(Erms, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(Erms, 0)))
        #print(genFeatures.shape)

        Erms_delta=librosa.feature.delta(Erms0)
        Erms_delta=np.transpose(Erms_delta)
        genFeatures = np.hstack((genFeatures, np.amin(Erms_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(Erms_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.median(Erms_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(Erms_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.std(Erms_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.var(Erms_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(Erms_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(Erms_delta, 0)))
        #print(genFeatures.shape)

        cent0 = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=hop_length*2, hop_length=hop_length)
        cent=np.transpose(cent0)
        genFeatures = np.hstack((genFeatures, np.amin(cent, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(cent, 0)))
        genFeatures = np.hstack((genFeatures, np.median(cent, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(cent, 0)))
        genFeatures = np.hstack((genFeatures, np.std(cent, 0)))
        genFeatures = np.hstack((genFeatures, np.var(cent, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(cent, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(cent, 0)))
        #print(genFeatures.shape)

        cent_delta=librosa.feature.delta(cent0)
        cent_delta=np.transpose(cent_delta)
        genFeatures = np.hstack((genFeatures, np.amin(cent_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(cent_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.median(cent_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(cent_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.std(cent_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.var(cent_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(cent_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(cent_delta, 0)))
        #print(genFeatures.shape)
        #Each frame of a magnitude spectrogram is normalized and treated as a distribution over frequency bins, from which the mean (centroid) is extracted per frame.

        ############### pitch at certain frame
        pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr, fmin=75, fmax=8000, n_fft=hop_length*2, hop_length=hop_length)
        p=[pitches[magnitudes[:,i].argmax(),i] for i in range(0,pitches.shape[1])]
        pitch0=np.array(p)   #shape (305,)
        pitch=np.transpose(pitch0)
        genFeatures = np.hstack((genFeatures, np.amin(pitch, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(pitch, 0)))
        genFeatures = np.hstack((genFeatures, np.median(pitch, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(pitch, 0)))
        genFeatures = np.hstack((genFeatures, np.std(pitch, 0)))
        genFeatures = np.hstack((genFeatures, np.var(pitch, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(pitch, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(pitch, 0)))
        #print(genFeatures.shape)

        pitch_delta=librosa.feature.delta(pitch0)
        pitch_delta=np.transpose(pitch_delta)
        genFeatures = np.hstack((genFeatures, np.amin(pitch_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.amax(pitch_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.median(pitch_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.mean(pitch_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.std(pitch_delta, 0)))
        genFeatures = np.hstack((genFeatures, np.var(pitch_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.skew(pitch_delta, 0)))
        genFeatures = np.hstack((genFeatures, st.kurtosis(pitch_delta, 0)))
    except:
        return None
    #print(genFeatures.shape)    #272
    return genFeatures

In [5]:
def str_to_int(input_np):
    output_np = []
    for x in np.nditer(input_np):
        if x == 'H':
            x = 0
        elif x == 'A':
            x = 1
        elif x == 'N':
            x = 2
        else:
            x = 3
        output_np.append(x)
    output_np = np.array(output_np)
    output_np = np.reshape(output_np, (len(output_np), 1) )
    
    return output_np

def float_compatible(input_np):

    x = np.where(input_np >= np.finfo(np.float32).max)
    for index in range(0, len(x[0])):
        x_position = x[0][index]
        y_position = x[1][index]
        
        input_np[x_position, y_position] = 0.0

In [6]:
# these funstion will generate feature set for feature selection from training
def function_FeatureExtract(path, classY):
    count = 0
    number_of_elements = len(os.listdir(path))
    
    extension = '.wav'
    flag_start_all = 0
    flag_Y_start = 0
    file_counter = 1
    All = np.array([])
    ClassLabel = np.array([])
    for root, dirs_list, files_list in os.walk(path):
        for file_name in files_list:
            count += 1
            update_progress(count / number_of_elements )

            # if os.path.splitext(file_name)[-1] == extension:
            # print(file_name)
            audiofile = os.path.join(root, file_name)

            audio, s_rate = librosa.load(audiofile, sr=sample_rate)

            segment_start_flag = 0
            start_seg = 0
            while (start_seg + segment_length) < len(audio):
                flag = 1
                sound1 = audio[start_seg:(start_seg + segment_length)]

                featureSet = function_FeatureExtractfromSinglewindow(
                    sound1, hop_length, sample_rate)

                # featureSet = np.hstack((featureSet, classValue))

                if segment_start_flag == 0:
                    SegAllFeat = featureSet
                    segment_start_flag = 1
                else:
                    SegAllFeat = np.vstack((SegAllFeat, featureSet))

                if flag_Y_start == 0:
                    ClassLabel = [classY]
                    flag_Y_start = 1
                else:
                    ClassLabel = np.vstack((ClassLabel, [classY]))

                ################################ end of class value identify
                start_seg = start_seg + overlapping
            
            if segment_start_flag == 1:
                # print(SegAllFeat.shape)
                SegAllFeat = np.nan_to_num(SegAllFeat)
                float_compatible(SegAllFeat)
                SegAllFeat = normalize(SegAllFeat, norm='l2', axis=0)

            #print(SegAllFeat.shape)
            if flag_start_all == 0:
                All = SegAllFeat
                flag_start_all = 1
            else:
                All = np.vstack((All, SegAllFeat))


    return All, ClassLabel

In [7]:
'''
Extract specified amount of features from an audio file
'''
def function_FeatureExtract1(audiofile, NumofFeatures):
    extension = '.wav'
    flag_start_all = 0
    flag_Y_start = 0
    All = np.array([])
    NumofFeaturetoUse = NumofFeatures #needs to be reassigned, takes two parameters
    ListOfFrame2Vec = np.empty((0, frame_number, NumofFeaturetoUse))
    audio, s_rate = librosa.load(audiofile, sr=sample_rate)
    segment_start_flag = 0
    start_seg = 0
    while (start_seg + segment_length) < len(audio):
        flag = 1
        sound1 = audio[start_seg:(start_seg + segment_length)]

        featureSet = function_FeatureExtractfromSinglewindow(sound1, hop_length, sample_rate)

        if segment_start_flag == 0:
            SegAllFeat = featureSet
            segment_start_flag = 1
        else:
            SegAllFeat = np.vstack((SegAllFeat, featureSet))

        start_seg = start_seg + overlappiong

    if segment_start_flag == 1:
        #print(SegAllFeat.shape)
        SegAllFeat = normalize(SegAllFeat, norm='l2', axis=0)

    #print(SegAllFeat.shape)
    if flag_start_all == 0:
        All = SegAllFeat
        flag_start_all = 1
    else:
        All = np.vstack((All, SegAllFeat))

    return All

In [8]:
def obtain_new_feature_vector(feature_raw, feature_vector, label, label_vector):
    
    # feature_raw = the path in which the raw files (images, audios) are saved.
    # feature_vector = the npy file consisting of all the feature vectors in the class
    # label = the label of the class, should be numerical,  a single number
    # label_vector = just an npy array of the labels
    
    if os.path.exists(feature_vector):
        print('Deleting old feature vector npy ...')
        os.remove(feature_vector)
        
    if os.path.exists(label_vector):
        print('Deleting old label vector npy ...')
        os.remove(label_vector)
    
    featureSet, Label = function_FeatureExtract(feature_raw, label)  # change here
    
    np.save(feature_vector, featureSet)
    np.save(label_vector, Label)


    vec = featureSet/np.float32(255)
    # label = Label.astype(np.int32)  # not required
    
    return np.array(vec), np.array(label)

In [9]:
def load_old_feature_vector(feature_vector, label_vector):
    
    # feature_raw = the path in which the raw files (images, audios) are saved.
    # feature_vector = the npy file consisting of all the feature vectors in the class
    # label = the label of the class, should be numerical,  a single number
    # label_vector = just an npy array of the labels
    vec = np.array([])
    ret_label = np.array([])
    
    if not os.path.exists(feature_vector):
        print('Unable to find old feature vector npy ...')
        
    elif not os.path.exists(label_vector):
        print('Unable to find old label vector npy ... ')

    else:
        vec = np.load(feature_vector)
        ret_label = np.load(label_vector)

    return np.array(vec), np.array(label)

In [67]:
vec_h_test, label_h_test = obtain_new_feature_vector(h_test, h_feature_vector_test, 0, h_label_vector_test)

Progress: [####################################################################################################] 100.0%


In [68]:
vec_h, label_h = obtain_new_feature_vector(h_directory, h_feature_vector, 0, h_label_vector)

Progress: [####################################################################################################] 100.0%


In [10]:
vec_a_test, label_a_test = obtain_new_feature_vector(a_test, a_feature_vector_test, 1, a_label_vector_test)

Progress: [####################################################################################################] 100.0%


In [11]:
vec_a, label_a = obtain_new_feature_vector(a_directory, a_feature_vector, 1, a_label_vector)

Progress: [####################################################################################################] 100.0%


In [12]:
vec_n_test, label_n_test = obtain_new_feature_vector(n_test, n_feature_vector_test, 2, n_label_vector_test)

Progress: [####################################################################################################] 100.0%


In [10]:
vec_n, label_n = obtain_new_feature_vector(n_directory, n_feature_vector, 2, n_label_vector)

Progress: [####################################################################################################] 100.0%


In [11]:
vec_s, label_s = obtain_new_feature_vector(s_directory, s_feature_vector, 3, s_label_vector)

Progress: [####################################################################################################] 100.0%


In [12]:
vec_s_test, label_s_test = obtain_new_feature_vector(s_test, s_feature_vector_test, 3, s_label_vector_test)

Progress: [####################################################################################################] 100.0%


In [14]:
vec_h = np.load('Features_48//h_feature_vector_48.npy')
label_h = np.load('Features_48//h_label_vector_48.npy')

vec_h_test = np.load('Features_48//h_feature_vector_test_48.npy')
label_h_test = np.load('Features_48//h_label_vector_test_48.npy')

vec_a = np.load('Features_48//a_feature_vector_48.npy')
label_a = np.load('Features_48//a_label_vector_48.npy')

vec_a_test = np.load('Features_48//a_feature_vector_test_48.npy')
label_a_test = np.load('Features_48//a_label_vector_test_48.npy')

vec_n = np.load('Features_48//n_feature_vector_48.npy')
label_n = np.load('Features_48//n_label_vector_48.npy')

vec_n_test = np.load('Features_48//n_feature_vector_test_48.npy')
label_n_test = np.load('Features_48//n_label_vector_test_48.npy')

vec_s = np.load('Features_48//s_feature_vector_48.npy')
label_s = np.load('Features_48//s_label_vector_48.npy')

vec_s_test = np.load('Features_48//s_feature_vector_test_48.npy')
label_s_test = np.load('Features_48//s_label_vector_test_48.npy')

In [16]:
featureSet_val=np.vstack((vec_h_test, vec_a_test, vec_n_test, vec_s_test))
Label_val=np.vstack((label_h_test, label_a_test, label_n_test, label_s_test))

featureSet=np.vstack((vec_h, vec_a, vec_n, vec_s))
Label=np.vstack((label_h, label_a, label_n, label_s))

In [17]:
def feature_ranking():
    # featureSet0, Label0 = function_FeatureExtract('train/n', 0)         #### change here
    # featureSet1, Label1 = function_FeatureExtract('train/p', 1)          #### change here
    
    indices_filename = 'Features_48//indices_filename.npy'

    ############################################################ feature selection
    X, X_test, Y, Y_test= train_test_split(featureSet, Label, test_size=0.2, random_state=42)
    seed = 7
    num_trees = 100
    #max_features = 200

    forest = RandomForestClassifier(n_estimators=num_trees)

    forest.fit(X,Y)

    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    np.save(indices_filename, indices)
    
    return indices

    ############## whole code is random forest based feature selection
indices = feature_ranking()

Feature ranking:
1. feature 13 (0.018338)
2. feature 39 (0.014769)
3. feature 26 (0.014083)
4. feature 225 (0.010583)
5. feature 49 (0.010494)
6. feature 36 (0.010459)
7. feature 227 (0.010414)
8. feature 0 (0.009939)
9. feature 226 (0.009161)
10. feature 41 (0.007981)
11. feature 28 (0.007714)
12. feature 242 (0.007456)
13. feature 228 (0.007446)
14. feature 14 (0.007307)
15. feature 243 (0.007291)
16. feature 240 (0.007160)
17. feature 23 (0.007118)
18. feature 15 (0.006906)
19. feature 32 (0.006904)
20. feature 6 (0.006738)
21. feature 34 (0.006636)
22. feature 40 (0.006628)
23. feature 224 (0.006608)
24. feature 45 (0.006597)
25. feature 229 (0.006559)
26. feature 236 (0.006490)
27. feature 27 (0.006393)
28. feature 65 (0.006089)
29. feature 5 (0.006015)
30. feature 47 (0.005992)
31. feature 258 (0.005937)
32. feature 42 (0.005880)
33. feature 44 (0.005722)
34. feature 29 (0.005693)
35. feature 4 (0.005377)
36. feature 237 (0.005354)
37. feature 31 (0.005137)
38. feature 211 (0.005

In [ ]:
def function_FeatureExtract_withword2vec(path, classLabel):
    print('Processing ' + path)
    
    extension = '.wav'
    flag_start_all = 0
    flag_Y_start = 0
    file_counter = 1
    ListOfFrame2Vec = np.empty((0, frame_number, NumofFeaturetoUse))
    LabelOfTotalFrame2Vec = np.empty((0, 1, 1))
    
    number_of_elements = len(os.listdir(path))
    i = 0

    for root, dirs_list, files_list in os.walk(path):
        for file_name in files_list:
            update_progress(i / number_of_elements)
            i += 1
            # if os.path.splitext(file_name)[-1] == extension:
            audiofile = os.path.join(root, file_name)

            audio, s_rate = librosa.load(audiofile, sr=sample_rate)
            # print(file_name)
            segment_start_flag = 0
            start_seg = 0
  
            while (start_seg + segment_length) < len(audio):

                sound1 = audio[start_seg:(start_seg + segment_length)]

                featureSet = function_FeatureExtractfromSinglewindow(
                    sound1, hop_length, sample_rate)

                # featureSet = np.hstack((featureSet, classValue))

                if segment_start_flag == 0:
                    SegAllFeat = featureSet
                    segment_start_flag = 1
                else:
                    SegAllFeat = np.vstack((SegAllFeat, featureSet))

                ################################ end of class value identify
                start_seg = start_seg + overlapping

            if segment_start_flag == 1:
                print(SegAllFeat.shape)
                SegAllFeat = np.nan_to_num(SegAllFeat).astype(np.float32)
                float_compatible(SegAllFeat)
                
                SegAllFeat = normalize(SegAllFeat, norm='l2', axis=0)

            # here you are doing feature selection
            SegFeatX = SegAllFeat[:, indices[0:NumofFeaturetoUse]]
            SegFeatX = SegFeatX[0:frame_number]

            ListOfFrame2Vec = np.append(
                ListOfFrame2Vec, array([SegFeatX]), axis=0)

            #print(ListOfFrame2Vec.shape)

            if classLabel == 1:
                LabelOfTotalFrame2Vec = np.append(
                    LabelOfTotalFrame2Vec, np.array([1]))
            elif classLabel == 2:
                LabelOfTotalFrame2Vec = np.append(
                    LabelOfTotalFrame2Vec, np.array([2]))
            elif classLabel == 3:
                LabelOfTotalFrame2Vec = np.append(
                    LabelOfTotalFrame2Vec, np.array([3]))
            else:
                LabelOfTotalFrame2Vec = np.append(
                    LabelOfTotalFrame2Vec, np.array([0]))
        
    return ListOfFrame2Vec, LabelOfTotalFrame2Vec

In [ ]:
# Feature selection
try:
    indices = np.load('Features_48//indices_filename.npy')
else:
    indices = feature_ranking()
    np.save('indices_filename.npy', indices)

In [ ]:
h_directory = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Happy//'
h_feature_vector = 'Features//h_feature_vector_48.npy' 
h_label_vector = 'Features//h_label_vector_48.npy'
h_label = 'H'

a_directory = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Angry//'
a_feature_vector = 'Features//a_feature_vector_48.npy' 
a_label_vector = 'Features//a_label_vector_48.npy'
a_label = 'A'

n_directory = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Neutral//'
n_feature_vector = 'Features//n_feature_vector_48.npy' 
n_label_vector = 'Features//n_label_vector_48.npy'
n_label = 'N'

s_directory = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Sad//'
s_feature_vector = 'Features//s_feature_vector_48.npy' 
s_label_vector = 'Features//s_label_vector_48.npy'
s_label = 'S'

h_test = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Happy_test//'
h_feature_vector_test = 'Features//h_feature_vector_test_48.npy' 
h_label_vector_test = 'Features//h_label_vector_test_48.npy'
h_label_test = 'H'

a_test = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Angry_test//'
a_feature_vector_test = 'Features//a_feature_vector_test_48.npy' 
a_label_vector_test = 'Features//a_label_vector_test_48.npy'
a_label_test = 'A'

n_test = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Neutral_test//'
n_feature_vector_test = 'Features//n_feature_vector_test_48.npy' 
n_label_vector_test = 'Features//n_label_vector_test_48.npy'
n_label_test = 'N'

s_test = '..//Datasets//TRAINING_PAD_NOISE_REVERB//Sad_test//'
s_feature_vector_test = 'Features//s_feature_vector_test_48.npy' 
s_label_vector_test = 'Features//s_label_vector_test_48.npy'
s_label_test = 'S'

In [ ]:
################## train test set
featureSet_train0, Label_train0 = function_FeatureExtract_withword2vec(h_directory, 0)
np.save(h_feature_vector, featureSet_train0)
np.save(h_label_vector, Label_train0)

In [ ]:
featureSet_train1, Label_train1 = function_FeatureExtract_withword2vec(a_directory, 1)
np.save(a_feature_vector, featureSet_train1)
np.save(a_label_vector, Label_train1)

In [ ]:
featureSet_train2, Label_train2 = function_FeatureExtract_withword2vec(n_directory, 2)
np.save(n_feature_vector, featureSet_train2)
np.save(n_label_vector, Label_train2)

In [ ]:
featureSet_train3, Label_train3 = function_FeatureExtract_withword2vec(s_directory, 3)
np.save(s_feature_vector, featureSet_train3)
np.save(s_label_vector, Label_train3)

In [ ]:
featureSet_validation0, Label_validation0 = function_FeatureExtract_withword2vec(h_test, 0)
np.save(h_feature_vector_test, featureSet_validation0)
np.save(h_label_vector_test, Label_validation0)

In [ ]:
featureSet_validation1, Label_validation1 = function_FeatureExtract_withword2vec(a_test, 1)
np.save(a_feature_vector_test, featureSet_validation1)
np.save(a_label_vector_test, Label_validation1)

In [ ]:
featureSet_validation2, Label_validation2 = function_FeatureExtract_withword2vec(n_test, 2)
np.save(n_feature_vector_test, featureSet_validation2)
np.save(n_label_vector_test, Label_validation2)

In [ ]:
featureSet_validation3, Label_validation3 = function_FeatureExtract_withword2vec(s_test, 3)
np.save(s_feature_vector_test, featureSet_validation3)
np.save(s_label_vector_test, Label_validation3)

In [ ]:
'''
featureSet_val=np.vstack((featureSet_validation0, featureSet_validation1))
Label_val=np.append(Label_validation0, Label_validation1, axis=0)

featureSet=np.vstack((featureSet_train0, featureSet_train1))
Label=np.append(Label_train0, Label_train1, axis=0)
'''